# 02. Pattern Comparison: ReAct vs Plan-and-Execute

This notebook runs a simple simulation to compare orchestration cost.


In [ ]:
from __future__ import annotations

import json
import os
import math
import random
import statistics
from pathlib import Path


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists() and (candidate / 'main_langgraph.py').exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
random.seed(42)


def simulate_react(complexity: int, uncertainty: int) -> dict:
    loops = complexity + max(1, uncertainty // 2)
    llm_calls = loops * 2
    tool_calls = loops
    return {'llm_calls': llm_calls, 'tool_calls': tool_calls, 'loops': loops}


def simulate_plan_execute(complexity: int, uncertainty: int) -> dict:
    plan_steps = max(3, complexity)
    replans = 1 if uncertainty >= 8 else 0
    llm_calls = 2 + replans
    tool_calls = plan_steps
    return {'llm_calls': llm_calls, 'tool_calls': tool_calls, 'loops': 1 + replans}


In [ ]:
tasks = [
    {'name': 'deterministic research', 'complexity': 7, 'uncertainty': 3},
    {'name': 'semi-dynamic analysis', 'complexity': 8, 'uncertainty': 6},
    {'name': 'chaotic environment', 'complexity': 6, 'uncertainty': 9},
]

rows = []
for t in tasks:
    react = simulate_react(t['complexity'], t['uncertainty'])
    plan = simulate_plan_execute(t['complexity'], t['uncertainty'])
    rows.append((t['name'], react['llm_calls'], plan['llm_calls'], react['tool_calls'], plan['tool_calls']))

for row in rows:
    print(row)


In [ ]:
react_llm_avg = statistics.mean([r[1] for r in rows])
plan_llm_avg = statistics.mean([r[2] for r in rows])
print('avg_react_llm_calls =', react_llm_avg)
print('avg_plan_llm_calls =', plan_llm_avg)

assert plan_llm_avg < react_llm_avg
print('Comparison check passed: planning uses fewer LLM calls in this setup.')


## Interpretation

- ReAct is more adaptive.
- Plan-and-Execute often reduces LLM call count and control variance.
